In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
!pip install keras-tuner
!pip install tensorflow_addons
from sklearn.model_selection import RandomizedSearchCV
import tensorflow_addons as tfa
#from tensorflow.keras import mixed_precision
import keras_tuner as kt
from xgboost import XGBClassifier
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy(policy)
keras.utils.set_random_seed(9)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
model = keras.models.load_model('/content/drive/MyDrive/data_490/final_model')

In [3]:
injury_history = pd.read_csv('/content/drive/MyDrive/data_490/injury_data/injury_history')
bio_data = pd.read_csv('/content/drive/MyDrive/data_490/processed_data/bio_data.csv')
ts_data = pd.read_pickle('/content/drive/MyDrive/data_490/processed_data/thirty_day_timestep_df')
bio_data.player_age = bio_data['player_age'].str.split(' ').str[0]

In [4]:
#for layer in model.layers[-9]: # Skip first and last layer
#  base_model.add(layer)
base_model = keras.Model(inputs=model.input, outputs=model.layers[-10].output)

In [5]:
train_indexes = np.loadtxt('/content/drive/MyDrive/data_490/processed_data/train_indexes.csv')
valid_indexes = np.loadtxt('/content/drive/MyDrive/data_490/processed_data/valid_indexes.csv')
test_indexes = np.loadtxt('/content/drive/MyDrive/data_490/processed_data/test_indexes.csv')
bio_data = bio_data.fillna(-1)
injury_history = injury_history.fillna(-1)
ts_train = np.stack(ts_data.loc[train_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
ts_valid = np.stack(ts_data.loc[valid_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
ts_test = np.stack(ts_data.loc[test_indexes, 'time_series'].values).astype(np.float16)[:,:,:-1]
bio_train = bio_data.iloc[train_indexes, 2:].astype(np.float16)
bio_valid = bio_data.iloc[valid_indexes, 2:].astype(np.float16)
bio_test = bio_data.iloc[test_indexes, 2:].astype(np.float16)
injury_train = injury_history.iloc[train_indexes, 2:].astype(np.float16)
injury_valid = injury_history.iloc[valid_indexes, 2:].astype(np.float16)
injury_test = injury_history.iloc[test_indexes, 2:].astype(np.float16)
train_target =  ts_data.loc[train_indexes, 'injured'].astype(np.float16)
valid_target =  ts_data.loc[valid_indexes, 'injured'].astype(np.float16)
test_target = ts_data.loc[test_indexes, 'injured'].astype(np.float16)

In [6]:
test_output = base_model.predict([ts_test, injury_test, bio_test])
valid_output = base_model.predict([ts_valid, injury_valid, bio_valid])
train_output = base_model.predict([ts_train, injury_train, bio_train])

In [17]:
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, .05, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.7, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.7, 1.0, 0.1),
           'reg_alpha':np.arange(0,.4,.05),
           'n_estimators': [5000]}
xg = XGBClassifier()

In [ ]:
search = RandomizedSearchCV(estimator=xg,
                         param_distributions=params,
                         scoring='f1',
                         n_iter=50,
                         verbose=3,
                         cv=4)
search.fit(train_output, train_target,verbose=True, eval_set=[(valid_output, valid_target)], early_stopping_rounds=100)

Streaming output truncated to the last 5000 lines.
[35]	validation_0-error:0.016162
[36]	validation_0-error:0.016209
[37]	validation_0-error:0.016162
[38]	validation_0-error:0.016209
[39]	validation_0-error:0.016209
[40]	validation_0-error:0.015736
[41]	validation_0-error:0.015736
[42]	validation_0-error:0.015736
[43]	validation_0-error:0.015831
[44]	validation_0-error:0.015878
[45]	validation_0-error:0.015831
[46]	validation_0-error:0.015831
[47]	validation_0-error:0.015926
[48]	validation_0-error:0.015595
[49]	validation_0-error:0.015547
[50]	validation_0-error:0.015547
[51]	validation_0-error:0.015642
[52]	validation_0-error:0.015642
[53]	validation_0-error:0.015595
[54]	validation_0-error:0.015642
[55]	validation_0-error:0.015642
[56]	validation_0-error:0.015595
[57]	validation_0-error:0.015595
[58]	validation_0-error:0.015642
[59]	validation_0-error:0.015406
[60]	validation_0-error:0.0155
[61]	validation_0-error:0.015358
[62]	validation_0-error:0.015311
[63]	validation_0-error:0.0

In [ ]:
search_results = pd.DataFrame(search.cv_results_)
search_results

In [ ]:
search_results.to_csv('/content/drive/MyDrive/data_490/xgb_head_search_results.csv')

In [ ]:
#pred = xg.predict_proba(test_output)[:,1]
#high_risk = np.where(pred > np.percentile(pred, 99))
#normal_risk = np.where(pred < np.percentile(pred, 99))
#low_risk = np.where(pred < np.percentile(pred, 40))
#print(f'high risk injury rate: {np.mean(test_target.iloc[high_risk[0]])}')
#print(f'normal risk injury rate: {np.mean(test_target.iloc[normal_risk[0]])}')
#print(f'low risk injury rate: {np.mean(test_target.iloc[low_risk[0]])}')

In [ ]:
#train_pred = xg.predict_proba(train_output)[:,1]
#high_risk = np.where(train_pred > np.percentile(train_pred, 99))
#normal_risk = np.where((train_pred < np.percentile(train_pred, 99))
#low_risk = np.where(train_pred < np.percentile(train_pred, 40))
#print(f'high risk injury rate: {np.mean(train_target.iloc[high_risk[0]])}')
#print(f'normal risk injury rate: {np.mean(train_target.iloc[normal_risk[0]])}')
#print(f'low risk injury rate: {np.mean(train_target.iloc[low_risk[0]])}')

In [ ]:
#test_high_risk = np.where(pred > np.percentile(train_pred, 99))
#test_normal_risk = np.where(pred < np.percentile(train_pred, 99))
#test_low_risk = np.where(pred < np.percentile(train_pred, 50))
#print(f'high risk injury rate: {np.mean(test_target.iloc[test_high_risk[0]])}')
#print(f'normal risk injury rate: {np.mean(test_target.iloc[test_normal_risk[0]])}')
#print(f'low risk injury rate: {np.mean(test_target.iloc[test_low_risk[0]])}')

In [ ]:
#from sklearn import metrics
#predictions = np.zeros(len(test_target), dtype=int)
#predictions[test_high_risk[0]] = 1
#metrics.ConfusionMatrixDisplay.from_predictions(test_target, predictions)

In [ ]:
#print(metrics.recall_score(test_target, predictions))
#print(metrics.precision_score(test_target, predictions))
#metrics.f1_score(test_target, predictions)